In [1]:
import sys
from pathlib import Path


# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *

In [2]:
# Setup for performance metrics
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 30
SIZES = [5, 10, 25, 50]

# Max Heap

In [13]:
from typing import List

HEAP_VALUES = "((int - 0).(list.idx))"
# LEFT_CHILDREN = f"{{ parent, child : {HEAP_VALUES} | (some i, i2 : int | (@num:i2 = multiply[@num:i, 2]) and (i->parent + i2->child) in (list.idx) )}}"
#RIGHT_CHILDREN = f"{{ parent, child : {HEAP_VALUES} | (some i, i2 : int | (@num:i2 = add[1, multiply[@num:i, 2]]) and (i->parent + i2->child) in (list.idx))}}"


LEFT_CHILDREN = "{ parent, child : ((int - 0).(list.idx)) | (some i : int | (i->parent + (multiply[@num:i, 2])->child) in (list.idx) )}"
RIGHT_CHILDREN = "{ parent, child : ((int - 0).(list.idx)) | (some i : int | (i->parent + (add[1, multiply[@num:i, 2]])->child) in (list.idx) )}"
@hideAtom(selector=f'MaxHeap + list + (int - {HEAP_VALUES})')
@orientation(selector= LEFT_CHILDREN, directions=["left", "below"])
@inferredEdge(selector=LEFT_CHILDREN, name = "left")
@orientation(selector=RIGHT_CHILDREN, directions=["right", "below"])
@inferredEdge(selector=RIGHT_CHILDREN, name = "right")
class MaxHeap:
    """
    CLRS-style max heap storing integers.
    1-indexed: a[0] unused.
    """
    def __init__(self, data: List[int] = None):
        self.a: List[int] = [0]
        if data:
            self.a.extend(data)
        self.n = len(self.a) - 1
        if self.n > 1:
            self.build_max_heap()

    # index helpers
    @staticmethod
    def _parent(i: int) -> int: return i // 2
    @staticmethod
    def _left(i: int) -> int:   return 2 * i
    @staticmethod
    def _right(i: int) -> int:  return 2 * i + 1

    def _max_heapify(self, i: int) -> None:
        while True:
            l, r = self._left(i), self._right(i)
            largest = i
            if l <= self.n and self.a[l] > self.a[largest]:
                largest = l
            if r <= self.n and self.a[r] > self.a[largest]:
                largest = r
            if largest == i:
                break
            self.a[i], self.a[largest] = self.a[largest], self.a[i]
            i = largest

    def build_max_heap(self) -> None:
        for i in range(self.n // 2, 0, -1):
            self._max_heapify(i)

    # API
    def max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        return self.a[1]

    def extract_max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        m = self.a[1]
        self.a[1] = self.a[self.n]
        self.a.pop()
        self.n -= 1
        if self.n >= 1:
            self._max_heapify(1)
        return m

    def increase_key(self, i: int, key: int) -> None:
        if i < 1 or i > self.n:
            raise IndexError("index out of range")
        if key < self.a[i]:
            raise ValueError("new key is smaller than current key")
        self.a[i] = key
        while i > 1 and self.a[self._parent(i)] < self.a[i]:
            p = self._parent(i)
            self.a[i], self.a[p] = self.a[p], self.a[i]
            i = p

    def insert(self, key: int) -> None:
        self.n += 1
        self.a.append(float("-inf"))  # sentinel
        self.increase_key(self.n, key)

    def __len__(self) -> int:
        return self.n

    def __repr__(self) -> str:
        return f"MaxHeap({self.a[1:]})"



## Max Heap In CLRS

![max-heap](./img/max-heap.png)

In [14]:
h = MaxHeap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
diagram(h)


## Performance - Max Heap

In [15]:
STRUCTURE = "max_heap"
for size in SIZES:
    values = random.sample(range(1, 1000), size)
    h = MaxHeap(values)
    
    print(f"{STRUCTURE}({size} elements): Rendering with perf_iterations={PI}...")
    diagram(h, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI)
    sleep(2)

max_heap(5 elements): Rendering with perf_iterations=30...
max_heap(10 elements): Rendering with perf_iterations=30...
max_heap(25 elements): Rendering with perf_iterations=30...
max_heap(50 elements): Rendering with perf_iterations=30...
